In [1]:
import torch
from trainer import *
from torch.utils.data import DataLoader
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from models.CNN import StrainEnergyCANN_Ani
from sklearn.metrics import r2_score
from typing import Optional

# Предобработка данных

In [2]:
path_to_data = r"..\..\data\GoreTex"
# tensl_data_path = r"..\..\data\PDMS\Shear_pdms.csv"
# shear_data_path = r"..\..\data\PDMS\Tensile_pdms.csv"
# experiments_path = [compr_data_path, tensl_data_path, shear_data_path]

In [3]:
def get_list_of_paths_to_experiments_type(experiment="biaxial"):
    experiment_type_path = os.path.join(path_to_data, experiment)
    l = []
    for file in os.listdir(experiment_type_path):
        l.append(os.path.join(experiment_type_path, file))
    return l

def load_and_extract(file_path, experiment_type):
    df = pd.read_csv(file_path)
    df['experiment_type'] = experiment_type
    return df[['lambda_clamps_X', 'lambda_clamps_Y', 'mean_stress_x_mpa', 'mean_stress_y_mpa', 'experiment_type']]

get_list_of_paths_to_experiments_type()

['..\\..\\data\\GoreTex\\biaxial\\GoreTex080mm100_100.csv',
 '..\\..\\data\\GoreTex\\biaxial\\GoreTex089mm075_100.csv',
 '..\\..\\data\\GoreTex\\biaxial\\GoreTex090mm050_100.csv',
 '..\\..\\data\\GoreTex\\biaxial\\GoreTex090mm100_050.csv',
 '..\\..\\data\\GoreTex\\biaxial\\GoreTex090mm100_075.csv',
 '..\\..\\data\\GoreTex\\biaxial\\GoreTex096mm033_100.csv',
 '..\\..\\data\\GoreTex\\biaxial\\GoreTex096mm100_033.csv']

In [4]:
experiment = "biaxial"
experiments_path = get_list_of_paths_to_experiments_type(experiment)
data_frames = [load_and_extract(file, file[-11:-4]) for file in experiments_path]

# print(data_frames)

df = pd.concat(data_frames, ignore_index=True)
thinned_data_frames = []
num_points = 90

sampled_df_list = []

for df in data_frames:
    indices = np.linspace(10, len(df) - 1, num_points, dtype=int)
    # df[1] = df[1] / 10**6
    sampled_df = pd.DataFrame(df.iloc[indices].copy())
    # print(type(sampled_df))
    sampled_df['lambdas'] = list(zip(sampled_df['lambda_clamps_X'], sampled_df['lambda_clamps_Y']))
    sampled_df['stresses'] = list(zip(sampled_df['mean_stress_x_mpa'], sampled_df['mean_stress_y_mpa']))
    sampled_df_list.append(sampled_df)
    # thinned_df = df.iloc[::len(df) // 20, :]  # Выбор каждого 45-го значения
    # thinned_data_frames.append(thinned_df)
data_frames = sampled_df_list
data_frames[2]
# sampled_df_list[0]


,lambda_clamps_X,lambda_clamps_Y,mean_stress_x_mpa,mean_stress_y_mpa,experiment_type,lambdas,stresses
10,1.028571,1.057143,0.001607,0.027608,050_100,"(1.0285714285714285, 1.0571428571428572)","(0.0016072188773338, 0.0276077712634154)"
11,1.031429,1.062857,0.002074,0.034651,050_100,"(1.0314285714285714, 1.062857142857143)","(0.002073509042369, 0.0346511095457778)"
12,1.034286,1.068571,0.002591,0.043432,050_100,"(1.0342857142857145, 1.0685714285714285)","(0.0025909158043805, 0.0434319165009779)"
13,1.037143,1.074286,0.003248,0.052552,050_100,"(1.0371428571428571, 1.0742857142857145)","(0.0032484267440479, 0.0525524997351983)"
14,1.040000,1.080000,0.003959,0.062187,050_100,"(1.04, 1.08)","(0.0039588978536403, 0.0621872863412342)"
...,...,...,...,...,...,...,...
95,1.014286,1.028571,-0.000333,-0.002870,050_100,"(1.0142857142857142, 1.0285714285714285)","(-0.0003331233728058, -0.0028698994360673)"
96,1.011429,1.022857,-0.001027,-0.006587,050_100,"(1.0114285714285711, 1.022857142857143)","(-0.001027261564481, -0.006587165520817)"
97,1.008571,1.017143,-0.001743,-0.009434,050_100,"(1.008571428571429, 1.0171428571428571)","(-0.0017429231925088, -0.0094336679450146)"
98,1.005714,1.011429,-0.004714,-0.025783,050_100,"(1.0057142857142858, 1.0114285714285711)","(-0.0047144159273535, -0.0257834317564309)"


In [5]:
I1_bx = lambda lam1, lam2: lam1**2 + lam2**2 + 1 / (lam1 * lam2)**2
I2_bx = lambda lam1, lam2: 1 / lam1**2 + 1 / lam2**2 + (lam1 * lam2)**2
I4_bx = lambda lam1, lam2: lam1**2  + math.cos(math.pi / 4)**2 + lam2 * math.sin(torch.pi / 4)**2
I5_bx = lambda lam1, lam2: lam1**2  + math.cos(math.pi / 4)**4 + lam2 * math.sin(torch.pi / 4)**4

F_bx = lambda lam1, lam2: ([lam1, 0, 0], [0, lam2, 0], [0, 0, 1 / (lam1 * lam2)])


In [6]:
mechanical_variables = {
    "I1": I1_bx,
    "I2": I2_bx,
    "I4": I4_bx,
    "I5": I5_bx,

    "F": F_bx,
    # "exp_type": [(lambda x: 1), (lambda x: 0)] # 1 - torsion&compression, 0 - shear
    # "torsion_compression": (lambda x: 1)
}

# calculate I1, I2, F from lambda (torsion&compression and shear)
for variable in mechanical_variables.keys():
    func_calc = mechanical_variables.get(variable)

    for data_frame in data_frames:
        data_frame[variable] = data_frame['lambdas'].apply(lambda lambdas: func_calc(lambdas[0], lambdas[0]))


In [7]:
experiments = ['Compression', 'Tensile', 'Shear']
combined_data = pd.concat(data_frames).reset_index(drop=True, inplace=False)
# combined_data.columns = ['lambda1', 'P_experimental', 'I1', 'I2', 'F', 'experiment_type']
combined_data.pop("lambdas")
combined_data.pop("stresses")
experiment_type = combined_data.pop("experiment_type")
combined_data["experiment_type"] = experiment_type
combined_data

# combined_data = combined_data["lambda_clamps_X", "lambda_clamps_Y",	"mean_stress_x_mpa", "mean_stress_y_mpa", "experiment_type" 

,lambda_clamps_X,lambda_clamps_Y,mean_stress_x_mpa,mean_stress_y_mpa,I1,I2,I4,I5,F,experiment_type
0,1.057143,1.057143,0.002260,0.026224,3.035794,3.038548,2.146122,1.631837,"([1.0571428571428572, 0, 0], [0, 1.05714285714...",100_100
1,1.062857,1.062857,0.002675,0.033031,3.042941,3.046580,2.161094,1.645380,"([1.062857142857143, 0, 0], [0, 1.062857142857...",100_100
2,1.068571,1.068571,0.003215,0.039706,3.050673,3.055361,2.176131,1.658988,"([1.0685714285714285, 0, 0], [0, 1.06857142857...",100_100
3,1.074286,1.074286,0.003783,0.047987,3.058974,3.064891,2.191233,1.672661,"([1.0742857142857145, 0, 0], [0, 1.07428571428...",100_100
4,1.080000,1.080000,0.004571,0.057191,3.067830,3.075167,2.206400,1.686400,"([1.08, 0, 0], [0, 1.08, 0], [0, 0, 0.85733882...",100_100
...,...,...,...,...,...,...,...,...,...,...
625,1.028571,1.007143,0.000828,0.020486,3.009352,3.009710,2.072245,1.565102,"([1.0285714285714285, 0, 0], [0, 1.02857142857...",100_033
626,1.022857,1.005714,-0.000208,0.008024,3.006040,3.006225,2.057665,1.551951,"([1.022857142857143, 0, 0], [0, 1.022857142857...",100_033
627,1.017143,1.004286,-0.001441,-0.000880,3.003429,3.003507,2.043151,1.538865,"([1.0171428571428571, 0, 0], [0, 1.01714285714...",100_033
628,1.011429,1.002857,-0.005595,-0.013425,3.001538,3.001562,2.028702,1.525845,"([1.0114285714285711, 0, 0], [0, 1.01142857142...",100_033


In [8]:
# new_combined_data = pd.DataFrame(columns=["lambdas, 

In [9]:
from torch.utils.data import DataLoader, random_split, Dataset, TensorDataset
import copy 

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        # self.features = [dataframe[0],dataframe[2], dataframe[3], dataframe[4], dataframe[5]]
        # self.targets  = dataframe[1]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = copy.deepcopy([*self.data.iloc[idx]])
        target1 = features.pop(2)
        target2 = features.pop(2)
        target = torch.tensor([target1, target2])
        return features, target

    def to_tensor(self):
        for column in self.data.columns:
            if column != "experiment_type":
                self.data[column] = self.data[column].apply(
                    lambda x: torch.tensor(x, dtype=torch.float32)).copy()


In [10]:
start = 0
end = -1
train_dataset = CustomDataset(combined_data[start:end].copy())
test_dataset = CustomDataset(combined_data.copy())
f, t = train_dataset[1]
# lam, i1, i2, F, exp_type = f
print(f)
print(t)

[1.062857142857143, 1.062857142857143, 3.042941421104298, 3.0465795923987544, 2.161093877551021, 1.6453795918367349, ([1.062857142857143, 0, 0], [0, 1.062857142857143, 0], [0, 0, 0.8852179442710139]), '100_100']
tensor([0.0027, 0.0330], dtype=torch.float64)


In [11]:
def init_loaders(experiments:Optional[str]=["Shear", "Tensile", "Comression"]):
    if type(experiments) == str:
        experiments = [experiments]
    elif type(experiments) == list:
        df = pd.concat([combined_data[combined_data["experiment_type"] == experiment] for experiment in experiments]).reset_index(drop=True, inplace=False)
    else:
        df=combined_data
    train_dataset = CustomDataset(df.copy())
    test_dataset = CustomDataset(combined_data.copy())
    
    train_dataset.to_tensor()
    test_dataset.to_tensor()
    
    train_data_loader = DataLoader(
                            train_dataset,
                            shuffle=True,
                            # num_workers=1,
                            pin_memory=False
    )
    test_data_loader = DataLoader(
                            test_dataset,
                            shuffle=False,
                            # num_workers=1,
                            pin_memory=False
    )

    return train_data_loader, test_data_loader 

In [12]:
train_data_loader, test_data_loader = init_loaders(None)
train_data_loader.dataset.data

,lambda_clamps_X,lambda_clamps_Y,mean_stress_x_mpa,mean_stress_y_mpa,I1,I2,I4,I5,F,experiment_type
0,tensor(1.0571),tensor(1.0571),tensor(0.0023),tensor(0.0262),tensor(3.0358),tensor(3.0385),tensor(2.1461),tensor(1.6318),"[[tensor(1.0571), tensor(0.), tensor(0.)], [te...",100_100
1,tensor(1.0629),tensor(1.0629),tensor(0.0027),tensor(0.0330),tensor(3.0429),tensor(3.0466),tensor(2.1611),tensor(1.6454),"[[tensor(1.0629), tensor(0.), tensor(0.)], [te...",100_100
2,tensor(1.0686),tensor(1.0686),tensor(0.0032),tensor(0.0397),tensor(3.0507),tensor(3.0554),tensor(2.1761),tensor(1.6590),"[[tensor(1.0686), tensor(0.), tensor(0.)], [te...",100_100
3,tensor(1.0743),tensor(1.0743),tensor(0.0038),tensor(0.0480),tensor(3.0590),tensor(3.0649),tensor(2.1912),tensor(1.6727),"[[tensor(1.0743), tensor(0.), tensor(0.)], [te...",100_100
4,tensor(1.0800),tensor(1.0800),tensor(0.0046),tensor(0.0572),tensor(3.0678),tensor(3.0752),tensor(2.2064),tensor(1.6864),"[[tensor(1.0800), tensor(0.), tensor(0.)], [te...",100_100
...,...,...,...,...,...,...,...,...,...,...
625,tensor(1.0286),tensor(1.0071),tensor(0.0008),tensor(0.0205),tensor(3.0094),tensor(3.0097),tensor(2.0722),tensor(1.5651),"[[tensor(1.0286), tensor(0.), tensor(0.)], [te...",100_033
626,tensor(1.0229),tensor(1.0057),tensor(-0.0002),tensor(0.0080),tensor(3.0060),tensor(3.0062),tensor(2.0577),tensor(1.5520),"[[tensor(1.0229), tensor(0.), tensor(0.)], [te...",100_033
627,tensor(1.0171),tensor(1.0043),tensor(-0.0014),tensor(-0.0009),tensor(3.0034),tensor(3.0035),tensor(2.0432),tensor(1.5389),"[[tensor(1.0171), tensor(0.), tensor(0.)], [te...",100_033
628,tensor(1.0114),tensor(1.0029),tensor(-0.0056),tensor(-0.0134),tensor(3.0015),tensor(3.0016),tensor(2.0287),tensor(1.5258),"[[tensor(1.0114), tensor(0.), tensor(0.)], [te...",100_033


# Обучение модели

In [13]:
# Переименуем столбцы для удобства
# data.columns = ['strain', 'measured_force', 'I1', 'I2', 'F', 'experiment_type', 'predicted_force']

# Удалим ненужные столбцы
# data = data[['strain', 'measured_force', 'experiment_type', 'predicted_force']]
def plot_results(data:pd.DataFrame, plot_name=""):
    # Преобразуем столбец с предсказанной силой в числовой формат
    # data['P_model_' + plot_name] = data['P_model_' + plot_name].apply(lambda x: float(str(x).strip('[]')))
    data['P_model' + plot_name] = data['P_model' + plot_name].apply(lambda x: float(str(x).strip('[]')))
    
    # Создадим графики для каждого типа эксперимента
    experiment_types = data['experiment_type'].unique()
    def plot_with_r2(data, experiment_types):
        r2_scores = []
        fig, axes = plt.subplots(1, len(experiment_types), figsize=(15, 6), sharey=False)
        
        for ax, experiment in zip(axes, experiment_types):
            subset = data[data['experiment_type'] == experiment]
            r2 = r2_score(subset['P_experimental'], subset['P_model' + plot_name])
            # r2 = r2_score(subset['P_experimental'], subset['P_model_' + plot_name])
            
            sns.scatterplot(data=subset, x='lambda', y='P_experimental', label='P_experimental', ax=ax)
            # sns.lineplot(data=subset, x='lambda', y='P_model_' + plot_name, label='P_model', color='orange', ax=ax)
            sns.lineplot(data=subset, x='lambda', y='P_model' + plot_name, label='P_model', color='orange', ax=ax)
            ax.set_title(f'Experiment Type: {experiment}\nR² = {r2:.2f}')
            ax.set_xlabel('Strain')
            ax.set_ylabel('Force (kPa)')
            r2_scores.append(r2)
        
        plt.tight_layout()
        plt.savefig(plot_name)
        plt.show()
        
        return r2_scores
    # Вызовем функцию для построения графиков с r2
    plot_with_r2(data, experiment_types)

In [ ]:
# experiments=[["Tensile", "Comression"], ["Tensile", "Shear"], ["Shear", "Comression"], "Shear", "Tensile", "Compression"]
# experiments = ["Tensile", "Comression", "Shear"]
# models = [StrainEnergyCANN_C, StrainEnergyCANN_polinomial3]
models = [StrainEnergyCANN_Ani]
path = r"C:\Users\User\PycharmProjects\data-driven-constitutive-modelling\src\CANN_torch\pretrained_models"
for model in models:


    # for idx, experiment in enumerate(experiments):
        train_data_loader, test_data_loader = init_loaders(None)
        name = "GoreTex_" + str(model.__name__)
        print("----------------------------------------------------------------------")
        # print(experiment)
        test_train = Trainer(
            plot_valid=False,
            epochs=5000,
            experiment_name=name,
            l2_reg_coeff=None,
            learning_rate=0.0001,
            checkpoint=None,
            model=model
            # dtype = torch.float64
        )
        
        trained_model = test_train.train(train_data_loader, None, weighting_data=False)
    
        trained_model.eval()
        vpredictions = []
        vtargets = []
        for data in test_data_loader:
            features, target = data
            vpredictions.append(zip(trained_model(features).detach().numpy()))
        print(trained_model.get_potential())
        
        combined_data["P_model_" + name] = vpredictions
        plot_results(combined_data, name)
        combined_data.to_csv(os.path.join(os.path.join(path, str(name)), "data.csv"))
        combined_data.pop("P_model_" + name)
# trained_model = StrainEnergyCANN_C()


# print("R2:", r2_score_own(vtargets, vpredictions))
# plt.figure(figsize=(10, 5))
# plt.plot(vpredictions, label='P_pred', color='red')
# plt.plot(vtargets, label='P_true', color='black')
# plt.xlabel('lambda/gamma')
# plt.ylabel('P')
# plt.title('Predictions vs. Targets')
# plt.legend()

# plt.show()

----------------------------------------------------------------------
Directory pretrained_models\GoreTex_StrainEnergyCANN_Ani already exists
LOSS train 2.2761738916223364 valid 2.2761738916223364
------------------------------------------------------------------
Epoch [1/5000], Loss: 2.27617389, Test metric: 2.27617389
LOSS train 1.6182422980172781 valid 1.6182422980172781
------------------------------------------------------------------
Epoch [2/5000], Loss: 1.61824230, Test metric: 1.61824230
LOSS train 1.9656899992597154 valid 1.9656899992597154
LOSS train 3.015267745894472 valid 3.015267745894472
LOSS train 4.875664280898987 valid 4.875664280898987
LOSS train 7.399904443347265 valid 7.399904443347265
LOSS train 10.737167359155322 valid 10.737167359155322
LOSS train 14.897846016051277 valid 14.897846016051277
LOSS train 20.03174104236421 valid 20.03174104236421
LOSS train 25.413053539821078 valid 25.413053539821078
LOSS train 31.447461088876874 valid 31.447461088876874
LOSS train

$psi =   0.000000 * (I1 - 3) \\               + 0.026245 * (e^{  0.000000 * (I1 - 3)} - 1)\\               - 0.025429 * ln(1 - 0.052720 * (I1 - 3)) \\               + 0.000000 * (I1 - 3) ^ 2 \\               + 0.085033 * (e^{  0.000000 * (I1 - 3) ^ 2} - 1)\\               - 0.077004 * ln(1 - 0.046803 * (I1 - 3) ^ 2) \\                               + 0.006244 * (I2 - 3) \\               + 0.046553 * (e^{  0.060594 * (I2 - 3))} - 1)\\               - 0.123550* ln(1 - 0.126146 *  (I2 - 3)) \\               + 0.000000 * (I2 - 3) ^ 2 \\               + 0.132380 * (e^{  0.051048 *(I2 - 3) ^ 2)} - 1)\\               - 0.018332* ln(1 - 0.074555 * (I2 - 3) ^ 2)\\$

In [ ]:
best_model_path = r"C:\Users\User\PycharmProjects\data-driven-constitutive-modelling_old\src\CANN_torch\pretrained_models\PDMS_full\20240522_1306_2681.pth"
trained_model = StrainEnergyCANN_polinomial3()
trained_model.load_state_dict(torch.load(best_model_path))
trained_model.eval()
vpredictions = []
vtargets = []
for data in test_data_loader:
    features, target = data
    vpredictions.append(trained_model(features).detach().numpy())
print(trained_model.get_potential())
combined_data["P_model"] = vpredictions

plot_results(combined_data)